<a href="https://colab.research.google.com/github/genie0320/gpt_writer/blob/colab/main_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API_KEY 설정.
- colab 환경 : 오른쪽 메뉴 > 열쇠모양(유저시크릿) > OPENAI_API_KEY로 설정 후 해당 코드실행.

In [2]:
# colab
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# from dotenv import load_dotenv
# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")

print("Got the key" if openai_api_key else "Something goes wrong")

Got the key


In [3]:
# ___ setting ___
MODEL = "gpt-3.5-turbo"
MAX = 50
TEMP = 1.5
# _______________

# Init Lanchain llm-model

In [10]:
!pip install --quiet langchain langchain_core langchain-openai

In [11]:
# export OPENAI_API_KEY=openai_api_key
# set OPENAI_API_KEY=openai_api_key # on windows
# %env OPENAI_API_KEY=openai_api_key # in colab

In [12]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI(api_key=openai_api_key)
chat_model = ChatOpenAI(api_key=openai_api_key)

# Lanchain prompt Model.
The LLM send / returns a **string**, while the ChatModel send / returns a **message**

## LLM prompt

1.템플릿을 만들고 > 2.각 변수에 값을 할당한다. > 3.invoke로 보낸다.
```python
# from langchain.prompts import PromptTemplate # defrecated
from langchain_core.proppts import PromptTemplate
prompt = PromptTemplate.from_template( 'text {var}' + '\n\n text{var} +  {input})
new_prompt = prompt.format(  
    time = "old",  
    input = user_input  
)
llm.invoke(new_prompt)   
```

In [ ]:
# from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("hi! {text}")
prompt_simple = prompt.format(text="Nice to see you.")

print(f'question : {prompt_simple}')
print('***********************************************')
print(f'LLM answer : {llm.invoke(prompt_simple)}')
# print('***********************************************')
# print(f'prompt_simple answer : {chat_model.invoke(prompt_simple).content}') # 사실 chat으로 호출해도 달라지는건 딱히.

# 템플릿을 만들고
prompt_vari01 = (
    PromptTemplate.from_template('You are a {time} friend of mine')
    + " and live in {place}."
    + " We are a {relationship}"
    + " What are we doing in {input}?"
    + "\n\nAnswer me in just 1 sentence not any more."
)

prompt_vari02 = PromptTemplate.from_template(
    'You are a {time} friend of mine'
    + " and live in {place}."
    + " We are a {relationship}"
    + " What are we doing in {input}?"
    + "\n\nAnswer me in just 1 sentence not any more."
    )

# 변수를 짝지어주고
user_input = input('where shall we go? : ')

prompt_vari01 = prompt_vari01.format(
    time = "old",
    place = "New york",
    relationship = "long-time no see status",
    input = user_input
)
prompt_vari02 = prompt_vari02.format(
    time = "old",
    place = "New york",
    relationship = "long-time no see status",
    input = user_input
)

# 보낸다.
# print('***********************************************')
# print(f'given : {prompt_vari01}')
# print(f'prompt_vari01 answer : {chat_model.invoke(prompt_vari01).content}')
# print('***********************************************')
# print(f'given : {prompt_vari02}')
# print(f'prompt_vari02 answer : {chat_model.invoke(prompt_vari02).content}')

## Chat prompt

chat prompt도 구성방법은 llm(complettion)의 경우와 동일하다.   
1.템플릿을 만들고 > 2.각 변수에 값을 할당한다. > 3.invoke로 보낸다.

하지만 return 되서 돌아오는 값이 객체에 담겨오므로... **res.contets**로 꺼내써야 한다.
```python
# from langchain.prompts.chat import ChatPromptTemplate # defrecated
from langchain_core.prompts import ChatPromptTemplate

template = "You are my new friend. We met {input_place} for {output_activity}."
human_template = "{text}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template), # 값을 tuple로 전달.
    ("human", human_template),
])
chat_prompt.format_messages(input_place="in library", output_activity="being a study friend", text="Hey, Sweety!")
chat_model.invoke(prompt).content
```

In [31]:
# TEST 02
from langchain_core.prompts import ChatPromptTemplate

system_template = 'You are a {time} friend of mine. and live in {place} and now, We are a {relationship}. My friend would respond to me in one sentence, no more.'
human_template = "\n\nI want to go to {user_input}"

chat_template = ChatPromptTemplate.from_messages(  # v.[ from_template ] for llm.
    [
        ('system', system_template),
        ('human', 'Where shall we go, today?'), # 이건 아마도 샘플일 것 같은데...
        ('ai', 'As you go, my darling'),
        ('human', human_template),
    ]
)
# 아래와 같이 잘 말려 들어가 있는 걸 발견할 수 있다.
# HumanMessage(content='Where shall we go, today?')

user_input = input('where shall we go? : ')

prompt = chat_template.format_messages( # v. [ prompt.format ] for llm
    time = "old",
    place = "New york",
    relationship = "good mood",
    user_input = user_input
)

print(f'me : {prompt}')
print(f'Chat answer : {chat_model.invoke(prompt).content}')

where shall we go? : book store
me : [SystemMessage(content='You are a old friend of mine. and live in New york and now, We are a good mood. My friend would respond to me in one sentence, no more.'), HumanMessage(content='Where shall we go, today?'), AIMessage(content='As you go, my darling'), HumanMessage(content='\n\nI want to go to book store')]
Chat answer : Let's get lost in the pages of new stories.


# Issue

## format_instructions
prompt의 Output 의 형태를 정해줄 수도 있다. `\n\n{format_instructions}`  
아마도 몇개를 보여줘라... 할 때 어떻게 보여줄지를 알려주는 대목인 모양.
그런데 이건 output parser 에서도 정해줄 수 있고 물론 prompt자체에서 해결할 수도 있다. 왜 여기서 굳이?

In [ ]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}" # \n\n 는 왜 필요한거지?

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions()) # format_instructions에 따라서 출력해라 ? 굳이 왜 이렇게 하는지 모르겠음.
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "human_name"}) # 위의 teamplate > text 에 human_name을 넣어서 대답을 가져와라.

## schema
● 일단 포기.
schema와 template가 뭐가 다른 건지 모르겠음.  
> 둘다 일종의 foramtting을 위한 것인데...


문서에 따르자면, 순수하게 개발경험향상을 위해서 추가한 것이라고 한다. 하지만 다음과 같이 권고하고 있다.

- 단순한 메시지 전달을 위해서는 `Message`를 쓰고,
- 뭔가 변수를 통해 전달해야 할 것이 있는 경우에는 `MessageTemplate`을 활용하라고 한다.

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# llm.invoke("hi?")

# 단순한 프롬프트
# message = "Hi?"
# llm.invoke(message)

# # 복잡한 프롬프트
# template = 'You are a {time} friend of mine'
# human_template = 'And I want you to {activity}'
# ai_answered = 'Sure! why not!'

# prompt = (
#     SystemMessage(content = template) + HumanMessage(content = human_template) + AIMessage(content = ai_answered)
# )

# message01 = prompt.format_messages(
#     time = "old",
#     activity = "help my love life",
# )

# message02 = prompt.format_prompt(
#     time = "old",
#     activity = "help my love life",
# )

# prompt02 = SystemMessage(content="You are a nice pirate")
# prompt02_ = (
#     prompt02 + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
# )
# message03 = prompt02_.format_messages(input="i said hi")
# res = new_prompt.format_messages(input="i said hi")

# print(f'format_messages_prompt : {message01}')
# print(f'answer : {chat_model.invoke(message01)}')
# print('********************************************')
# print(f'format_prompt_prompt : {message02}')
# print(f'answer : {chat_model.invoke(message02)}')
# print('********************************************')
# print(f'given : {message03}')
# print(f'answer : {chat_model.invoke(message03)}')

# print(f'res : {res}')

In [ ]:
from langchain.chains import LLMChain
# from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=openai_api_key)
chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="korean")

--------------------------------------------------------------------------------

# Understanding output parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are new friend of mine"),
    ("user", "{user_input}")
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
ai_answered = chain.invoke({"user_input": "Hi, there!"})
print(f'New friend from LLMs : {ai_answered}')

# Few-shot prompt templates

Page link found | Content | As is link | to be link|
------------------|-----------|---------|-----------|
https://python.langchain.com/docs/modules/model_io/prompts | How to use few-shot examples with LLMs | https://python.langchain.com/docs/modules/model_io/few_shot_examples |https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a short {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="love")

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
    {
        "question": "Hi, honey?",
        "answer": """
Yes,
my love.
When did you get here?
""",
    },
    {
        "question": "We just arrived. What should we eat?",
        "answer": """
Now I'm very hungry, so I think I can eat anything.
""",
    },
    {
        "question": "Honey, What is your favorite animation these days?",
        "answer": """
You know I like "Love Live", right? A new season is on air these days.
""",
    },
    {
        "question": "Really? Then do you want to go see it together?",
        "answer": """
No, it's okay. You're busy, so let's wait until the next movie comes out and watch it together.
""",
    },
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

# For wordwrap for out cell.

In [ ]:
# # from ipywidgets import Textarea, widgets
# # from IPython.display import display

# output_to_display = res.content

# # print(~) > res = ~     >> res.contents 등 가공. output_to_display >>
# # 다시 위쪽의 코드에 반영.

# output_widget = Textarea(
#     value=output_to_display,
#     layout=widgets.Layout(width="100%", height='300px')  # Can change Height
# )
# display(output_widget)

function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)